In [ ]:
import json

with open("28-09_31-10_alldata.json", "r") as f:
    data = json.load(f)

In [ ]:
import pandas as pd
pd.DataFrame(data)

dfs = []
for result in data["results"]:
    serie = result["series"][0]
    for serie in result["series"]:
        df = pd.DataFrame(serie["values"], columns=serie["columns"])
        df.name = serie["name"]
        dfs.append(df)


In [ ]:
print(dfs[0].keys())

## Mesurements
- RMS : root mean square
- variance
- skewness : normalised third central moment
- kurtosis : normalised fourth central moment
- normalised sixth central moment
- PV : peak value

In [ ]:
df = dfs[0]
print(df.name)
df

In [ ]:
df["time"] = pd.to_datetime(df["time"], unit='ms')
df = df.drop(['client', "GW", "device", "mileage"], axis=1)
df

In [ ]:
df.loc[(df["time"] >= "2021-10-09") & (df["time"] <= "2021-10-12"), "label"] = "base"
df.loc[(df["time"] >= "2021-10-27 17:30") & (df["time"] <= "2021-10-29 00:00"), "label"] = "cuscinetti"

In [ ]:
df = df.set_index("time")

In [ ]:
dfb = df.loc[df["label"] == "base"].resample('H').mean()
dfc = df.loc[df["label"] == "cuscinetti"].resample('H').mean()

In [ ]:
dfb["label"] = "base"
dfc["label"] = "cuscinetti"
dfa = dfc.append(dfb)
dfa

In [ ]:
import seaborn as sns
# dfcf = dfcf.reset_index()
dfaf = dfa[['s_00', 's_01', 's_02',
       's_03', 's_04', 's_05', 's_06', 's_07', 's_08', 's_09', 's_10', 's_11',
       's_12', 's_13', 's_14', 's_15', 's_16', 's_17', 's_18', 's_19',
    'sonic_custom', 'sonic_rmslog', 'temp', 'vib_x_acc',
            'vib_x_kurt', 'vib_x_peak', 'vib_x_root',
            'vib_x_vel', 'vib_y_acc',
            'vib_y_kurt', 'vib_y_peak', 'vib_y_root', 'vib_y_vel', 'vib_z_acc',
            'vib_z_kurt', 'vib_z_peak',
            'vib_z_vel', "label"]]
sns.pairplot(dfaf, kind="kde", hue="label", corner=True)

In [ ]:
import umap
reducer = umap.UMAP()
embedding= reducer.fit_transform(dfa.drop("label", axis=1))

In [ ]:
from matplotlib import pyplot as plt
c=[sns.color_palette()[x] for x in dfa.label.map({"cuscinetti":0, "base":1})]
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(embedding[:, 0], embedding[:, 1],c=c)

In [ ]:
from sklearn.manifold import TSNE
reducer = TSNE()
embedding= reducer.fit_transform(dfa.drop("label", axis=1))
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(embedding[:, 0], embedding[:, 1],c=c)

In [ ]:
from sklearn.decomposition import PCA
reducer = PCA()
embedding= reducer.fit_transform(dfa.drop("label", axis=1))
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(embedding[:, 0], embedding[:, 1],c=c)

## Root mean square error

Requires a prediction to have an error

In [ ]:
# # df_loc = df.loc[df["time"] == "2021-10-27 17:32:32.494"]
# df_loc = df.loc[df["time"] >= "2021-10-31 12:05:52.937"]
# print(df_loc.mean())
# df_loc